<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>

<h1 align=center><font size = 5>Assignment: SQL Notebook for Peer Assignment</font></h1>

Estimated time needed: **60** minutes.

## Introduction
Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions 


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



In [ ]:
!pip install sqlalchemy==1.4.46


### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [1]:
%load_ext sql

In [2]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [ ]:
!pip install -q pandas==1.1.5

In [ ]:
%sql sqlite:///my_data1.db

In [4]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

**Note:This below code is added to remove blank rows from table**


In [ ]:
%sql create or replace table SPACEXTBL as select * from SPACEXTBL where Date is not null

## Tasks

Now write and execute SQL queries to solve the assignment tasks.

**Note: If the column names are in mixed case enclose it in double quotes
   For Example "Landing_Outcome"**

### Task 1




##### Display the names of the unique launch sites  in the space mission


In [23]:
cur.execute("select distinct(Launch_Site) from  SPACEXTBL")
sites = cur.fetchall()
for site in sites:
    print(site[0])

CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40



### Task 2


#####  Display 5 records where launch sites begin with the string 'CCA' 


In [24]:
cur.execute("select * from  SPACEXTBL where Launch_Site like 'CCA%' limit 5")
records = cur.fetchall()
for record in records:
    print(record)

('2010-04-06', '18:45:00', 'F9 v1.0  B0003', 'CCAFS LC-40', 'Dragon Spacecraft Qualification Unit', 0, 'LEO', 'SpaceX', 'Success', 'Failure (parachute)')
('2010-08-12', '15:43:00', 'F9 v1.0  B0004', 'CCAFS LC-40', 'Dragon demo flight C1, two CubeSats, barrel of Brouere cheese', 0, 'LEO (ISS)', 'NASA (COTS) NRO', 'Success', 'Failure (parachute)')
('2012-05-22', '07:44:00', 'F9 v1.0  B0005', 'CCAFS LC-40', 'Dragon demo flight C2', 525, 'LEO (ISS)', 'NASA (COTS)', 'Success', 'No attempt')
('2012-08-10', '00:35:00', 'F9 v1.0  B0006', 'CCAFS LC-40', 'SpaceX CRS-1', 500, 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt')
('2013-01-03', '15:10:00', 'F9 v1.0  B0007', 'CCAFS LC-40', 'SpaceX CRS-2', 677, 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt')


### Task 3




##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [32]:
cur.execute("select SUM(PAYLOAD_MASS__KG_) from SPACEXTBL where Customer = 'NASA (CRS)'")
total = cur.fetchall()[0][0]
print ('the total payload mass carried by boosters launched by NASA (CRS):', total)

the total payload mass carried by boosters launched by NASA (CRS): 45596


### Task 4




##### Display average payload mass carried by booster version F9 v1.1


In [35]:
cur.execute("select AVG(PAYLOAD_MASS__KG_) from SPACEXTBL where Booster_Version is 'F9 v1.1'")
avg = cur.fetchone()[0]
print ('the average payload mass carried by booster version F9 v1.1:', avg)

the average payload mass carried by booster version F9 v1.1: 2928.4


### Task 5

##### List the date when the first succesful landing outcome in ground pad was acheived.


_Hint:Use min function_ 


In [36]:
cur.execute("Select Min(Date) from SPACEXTBL Where Landing_Outcome is 'Success (ground pad)'")
first_success_data = cur.fetchone()[0]
print('the first successful landing in ground pad was on:', first_success_data)

the first successful landing in ground pad was on: 2015-12-22


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [41]:
cur.execute("Select distinct(Customer) from SPACEXTBL where Landing_Outcome is 'Success (drone ship)' and PAYLOAD_MASS__KG_ between 4000 and 6000")
boosters = cur.fetchall()
for b in boosters:
    print(b[0])

SKY Perfect JSAT Group
SES
SES EchoStar


### Task 7




##### List the total number of successful and failure mission outcomes


In [49]:
cur.execute("Select (Select count(*) from SPACEXTBL where Mission_Outcome like 'Success%') as Success, (Select count(*) from SPACEXTBL where Mission_Outcome like 'Failure%') as Failure")
mission_outcomes = cur.fetchone()
print('Success: ',mission_outcomes[0])
print('Failure: ',mission_outcomes[1])


Success:  100
Failure:  1


### Task 8



##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [53]:
cur.execute("Select distinct(Booster_Version) from SPACEXTBL where PAYLOAD_MASS__KG_=(Select max(PAYLOAD_MASS__KG_) from SPACEXTBL)")
booster_versions = cur.fetchall()
for i,v in enumerate(booster_versions):
    print(i+1,':',v[0])

1 : F9 B5 B1048.4
2 : F9 B5 B1049.4
3 : F9 B5 B1051.3
4 : F9 B5 B1056.4
5 : F9 B5 B1048.5
6 : F9 B5 B1051.4
7 : F9 B5 B1049.5
8 : F9 B5 B1060.2 
9 : F9 B5 B1058.3 
10 : F9 B5 B1051.6
11 : F9 B5 B1060.3
12 : F9 B5 B1049.7 


### Task 9


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 4, 2) as month to get the months and substr(Date,7,4)='2015' for year.**


In [67]:
cur.execute("Select substr(Date,6,2) from SPACEXTBL where Landing_Outcome is 'Failure (drone ship)' and substr(date,1,4)='2015'")
months = cur.fetchall()
for m in months:
    print(m[0])

10
04


### Task 10




##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [72]:
cur.execute("Select Landing_Outcome,count(*) from SPACEXTBL where Date between '2010-06-04' and '2017-03-20' group by Landing_Outcome order by count(*) desc")
outcomes = cur.fetchall()
for o in outcomes:
    print(o[0],o[1])

No attempt 10
Success (ground pad) 5
Success (drone ship) 5
Failure (drone ship) 5
Controlled (ocean) 3
Uncontrolled (ocean) 2
Precluded (drone ship) 1
Failure (parachute) 1


### Reference Links

* <a href ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20String%20Patterns%20-%20Sorting%20-%20Grouping/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : String Patterns, Sorting and Grouping</a>  

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html?origin=www.coursera.org">Hands-on Lab: Built-in functions</a>

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : Sub-queries and Nested SELECT Statements</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb">Hands-on Tutorial: Accessing Databases with SQL magic</a>

*  <a href= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb">Hands-on Lab: Analyzing a real World Data Set</a>




## Author(s)

<h4> Lakshmi Holla </h4>


## Other Contributors

<h4> Rav Ahuja </h4>


## Change log
| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2021-07-09 | 0.2 |Lakshmi Holla | Changes made in magic sql|
| 2021-05-20 | 0.1 |Lakshmi Holla | Created Initial Version |


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
